In [1]:
# Load libraries
import pandas as pa
import numpy as np

from ipywidgets import interact, widgets

from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.models import HoverTool
from bokeh.layouts import widgetbox
from bokeh.models.widgets import Slider
from bokeh.models import ColumnDataSource
from bokeh.io import push_notebook
output_notebook()

Loading BokehJS ...

In [2]:
# Build the plot
trainSource = ColumnDataSource(data={'x':[],'y':[]})
testSource = ColumnDataSource(data={'x':[],'y':[]})
predictSource = ColumnDataSource(data={'x':[],'y':[]})

p = figure(width=500, plot_height=500, title='Linear Regression', 
           x_range=[0, 1], y_range=[0, 3],
           x_axis_label='x', y_axis_label='y')
p.scatter(source=trainSource,x='x',y='y',color='green', size=10)
p.scatter(source=testSource,x='x',y='y',color='red', size=10)
p.line(source=predictSource,x='x',y='y',color='blue', line_width=2)
show(p, notebook_handle=True);

In [3]:
# The controls for the plot
@interact(train=widgets.IntSlider(min=2,max=50,step=1,value=10),
          test=widgets.IntSlider(min=0,max=50,step=1,value=0),
          deg=widgets.IntSlider(min=1,max=10,step=1,value=1),
          sigma=widgets.FloatSlider(min=0.01,max=2.0,step=0.1,value=0.01),
          mdeg=[1,2])
def f(train=5, test=0, deg=1, sigma=0.2, mdeg=1):
    # Generate data
    np.random.seed(1234)
    df = pa.DataFrame(columns=['x','yTrue', 'e', 'y'])
    df['x'] = np.random.uniform(size=[100])
    if mdeg == 1:
        df['yTrue'] = 1.7*df['x']+1
    elif mdeg == 2:
        df['yTrue'] = 10*(df['x']-0.5)**2+1
        
    df['e'] = np.random.normal(0, sigma, size=[100])
    df['y'] = df['yTrue'] + df['e']
    
    trainSource.data['x'] = df['x'][:train]
    trainSource.data['y'] = df['y'][:train]
    testSource.data['x'] = df['x'][50:(50+test)]
    testSource.data['y'] = df['y'][50:(50+test)]
    
    x = df['x'][:train]
    y = df['y'][:train]
    
    z = np.polyfit(x, y, deg=deg)
    p = np.poly1d(z)
    yHat = p(x)
    
    xGrid = np.linspace(0,1,100)
    yGrid = p(xGrid)

    predictSource.data['x'] = xGrid
    predictSource.data['y'] = yGrid
    
    push_notebook()

interactive(children=(IntSlider(value=10, description='train', max=50, min=2), IntSlider(value=0, description=…